In [1]:
import json
import pandas as pd
from tqdm.notebook import tqdm
pd.set_option('display.max_columns', None)

### 1. VectorDB 구축

#### Local DB 생성

In [2]:
attributes = pd.read_csv("../data/imaterialist-fashion-2020-fgvc7/attribute_specific.csv")
df = pd.read_csv("../data/imaterialist-fashion-2020-fgvc7/clothes_final2.csv")

In [3]:
data_read_f = list()

with open("../data/imaterialist-fashion-2020-fgvc7/upsert_vectors_fashion_fine_tuned.json", 'r') as file:
    for line in file:
        data = json.loads(line)
        data_read_f.append(data)

print(f"Successfully read {len(data_read_f)} fashion-fine-tuned CLIP embeddings from img_embeddings_fashion_fine_tuned.json")


Successfully read 97924 fashion-fine-tuned CLIP embeddings from img_embeddings_fashion_fine_tuned.json


In [4]:
df['vdb_id'] = df['ImageId'].astype(str) + "_" + df['entity_id'].astype(str)
df.drop(columns=['id'], inplace=True)

In [5]:
upsert_df_f = pd.DataFrame(data_read_f)

In [6]:
upsert_df_f.head(2)

,id,values,sparse_values,metadata
0,00000663ed1ff0c4e0132b9b9ac53f6e_0,"[0.18243342638015747, 0.7801597714424133, -0.0...","{'indices': [1035, 2053, 2171, 2240, 2315, 233...",{'img_path': 'imaterialist-fashion-2020-fgvc7/...
1,00000663ed1ff0c4e0132b9b9ac53f6e_1,"[0.046928003430366516, 0.9497855305671692, -1....","{'indices': [1035, 2053, 2058, 2098, 2171, 218...",{'img_path': 'imaterialist-fashion-2020-fgvc7/...


In [7]:
d = pd.merge(df, upsert_df_f, left_on='vdb_id', right_on='id')

In [8]:
# d.to_csv("local_db.csv", index=False)

In [9]:
d.head(2)

,entity_id,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds,second_AttributesIds,bbox,bbox_big,width,height,area,name,supercategory,AttributesNames,second_AttributesNames,vdb_id,id,values,sparse_values,metadata
0,0,00000663ed1ff0c4e0132b9b9ac53f6e,6068157 7 6073371 20 6078584 34 6083797 48 608...,5214,3676,6,"115,136,143,154,230,295,316,317",NaN,"[1163, 3923, 2839, 5212]","[1122, 3891, 2880, 5214]",1758,1323,2325834,pants,lowerbody,"symmetrical, regular (fit), low waist, maxi (l...",NaN,00000663ed1ff0c4e0132b9b9ac53f6e_0,00000663ed1ff0c4e0132b9b9ac53f6e_0,"[0.18243342638015747, 0.7801597714424133, -0.0...","{'indices': [1035, 2053, 2171, 2240, 2315, 233...",{'img_path': 'imaterialist-fashion-2020-fgvc7/...
1,1,00000663ed1ff0c4e0132b9b9ac53f6e,6323163 11 6328356 32 6333549 53 6338742 75 63...,5214,3676,0,"115,136,142,146,225,295,316,317",163,"[1212, 1371, 2394, 3978]","[1183, 1306, 2423, 4043]",1240,2737,3393880,"shirt, blouse",upperbody,"symmetrical, regular (fit), normal waist, abov...",shirt (collar),00000663ed1ff0c4e0132b9b9ac53f6e_1,00000663ed1ff0c4e0132b9b9ac53f6e_1,"[0.046928003430366516, 0.9497855305671692, -1....","{'indices': [1035, 2053, 2058, 2098, 2171, 218...",{'img_path': 'imaterialist-fashion-2020-fgvc7/...


In [10]:
metadata = d['metadata'].values
names = d['name'].values

In [11]:
metadata_new = list()

for n,m in zip(names, metadata):
    m['category'] = n
    metadata_new.append(m)

d['metadata'] = metadata_new

In [12]:
m

{'img_path': 'imaterialist-fashion-2020-fgvc7/cropped_images/ffffbf7014a9e408bfbb81a75bc70638_97923.jpg',
 'category': 'dress'}

In [13]:
d.head(2)

,entity_id,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds,second_AttributesIds,bbox,bbox_big,width,height,area,name,supercategory,AttributesNames,second_AttributesNames,vdb_id,id,values,sparse_values,metadata
0,0,00000663ed1ff0c4e0132b9b9ac53f6e,6068157 7 6073371 20 6078584 34 6083797 48 608...,5214,3676,6,"115,136,143,154,230,295,316,317",NaN,"[1163, 3923, 2839, 5212]","[1122, 3891, 2880, 5214]",1758,1323,2325834,pants,lowerbody,"symmetrical, regular (fit), low waist, maxi (l...",NaN,00000663ed1ff0c4e0132b9b9ac53f6e_0,00000663ed1ff0c4e0132b9b9ac53f6e_0,"[0.18243342638015747, 0.7801597714424133, -0.0...","{'indices': [1035, 2053, 2171, 2240, 2315, 233...",{'img_path': 'imaterialist-fashion-2020-fgvc7/...
1,1,00000663ed1ff0c4e0132b9b9ac53f6e,6323163 11 6328356 32 6333549 53 6338742 75 63...,5214,3676,0,"115,136,142,146,225,295,316,317",163,"[1212, 1371, 2394, 3978]","[1183, 1306, 2423, 4043]",1240,2737,3393880,"shirt, blouse",upperbody,"symmetrical, regular (fit), normal waist, abov...",shirt (collar),00000663ed1ff0c4e0132b9b9ac53f6e_1,00000663ed1ff0c4e0132b9b9ac53f6e_1,"[0.046928003430366516, 0.9497855305671692, -1....","{'indices': [1035, 2053, 2058, 2098, 2171, 218...",{'img_path': 'imaterialist-fashion-2020-fgvc7/...


#### PineconeDB에 업로드

- Pinecone upsert 형식에 맞게 내용을 변환
    - 각 카테고리별로 batch에 맞게 upsert

In [14]:
import os
from dotenv import load_dotenv
load_dotenv()

## pineconeDB에 upsert!!
from pinecone import Pinecone

pc_api_key = os.environ["PINECONE_API_KEY"]

pc = Pinecone(api_key=pc_api_key)
# index 개수 확인
# index_list = pc.list_indexes().indexes

# index description
index = pc.Index("fastcampus")
index.describe_index_stats()

{'dimension': 512,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

- Max size for an upsert request is 2MB. Recommended upsert limit is 100 vectors per request.

In [15]:
# 각 카테고리별로 나눠서 저장함
# 이는 이후 index를 개별로 저장하기 위함

# upsert!!
def create_batches(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

df_categories = dict()

for cat in tqdm(d['name'].unique()):
    part_df = d.loc[d['name']==cat]
    part_upserts = part_df[['id', 'values', 'sparse_values', 'metadata']].to_dict('records')
    # 100개 단위로 upsert
    df_categories[cat] = list(create_batches(part_upserts, 100))

  0%|          | 0/27 [00:00<?, ?it/s]

In [16]:
df_categories.keys()

dict_keys(['pants', 'shirt, blouse', 'jacket', 'top, t-shirt, sweatshirt', 'dress', 'shoe', 'glasses', 'skirt', 'bag, wallet', 'belt', 'headband, head covering, hair accessory', 'sock', 'hat', 'watch', 'glove', 'tights, stockings', 'sweater', 'tie', 'shorts', 'scarf', 'coat', 'vest', 'umbrella', 'cardigan', 'cape', 'jumpsuit', 'leg warmer'])

In [23]:
# df_categories['shoe'][0][0]

#### Upsert 형식

```json
{"id" : "0838a48a7b0bfa789a5181ab0e8f4ee2_3040", # 이미지 파일 이름 + entity ID
 "values" : [-0.08405803143978119, -0.7088879346847534, ...], # CLIP embeddings
 "sparse_values" : {
    "indices" : [1045, 1062, ...], # non-zero index
    "values" : [1.3038887977600098, 0.304147332906723, ...] # non-zero values
    },
"metadata" : {
    # 이미지 파일 path
    "img_path": "../data/imaterialist-fashion-2020-fgvc7/cropped_images/0838a48a7b0bfa789a5181ab0e8f4ee2_3040.jpg",
    "category": "coat"
} 
}

```

In [ ]:
# # upsert to 'fastcampus' index
# for cat, batches in df_categories.items():
#     print(cat)
#     for batch in tqdm(batches):
#         index.upsert(vectors=batch)

- 각 카테고리별로 개별 index에 저장

---

### 2. Text to image search

- CLIP embedding을 활용
    - text와 image를 한 vector space에 함께 표현되어 있음
    - 또한, fashion dataset에 맞도록 fine-tune되어, 일반 plain CLIP보다 현재 use case에 적합
    - Fine-tune된 데이터 역시 옷의 다양한 attribute을 바탕으로 트레이닝 [Fine-tune 훈련 데이터](https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41598-022-23052-9/MediaObjects/41598_2022_23052_Fig3_HTML.png?as=webp, "Fine-tune 훈련 데이터")

(출처 : Contrastive language and vision learning of general fashion concepts)


In [17]:
from PIL import Image
import os
import json
from tqdm import tqdm
import numpy as np
from image_utils import fetch_clip, draw_images
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer

In [18]:
from search_utils import gen_sparse_vector

In [19]:
from splade.splade.models.transformer_rep import Splade
from transformers import AutoTokenizer

splade_model_id = 'naver/splade-cocondenser-ensembledistil'

# splade = 'naver/splade-v3'
splade_model = Splade(splade_model_id, agg='max')
splade_model.to('cpu')  # move to GPU if possible
splade_model.eval()

splade_tokenizer = AutoTokenizer.from_pretrained(splade_model_id)

In [20]:
model, processor, tokenizer = fetch_clip(model_name="patrickjohncyh/fashion-clip")

In [21]:
def get_single_text_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors = "pt", padding=True)
    text_embeddings = model.get_text_features(**inputs)
    # convert the embeddings to numpy array
    embedding_as_np = text_embeddings.cpu().detach().numpy()
    return embedding_as_np.tolist()

In [27]:
input_text = "Green dress with blue dots, long sleeve"

d = get_single_text_embedding(input_text, model, tokenizer)

result = index.query(
    vector=d[0],
    top_k=5,
    filter={"category": {"$eq": "dress"}},
    include_metadata=True
)

paths = ['../data/'+i['metadata']['img_path'] for i in result.matches]

draw_images([Image.open(i) for i in paths])

In [ ]:
input_text = "nike"

# vans, nike, addidas

d = get_single_text_embedding(input_text, model, tokenizer)

result = index.query(
    vector=d[0],
    top_k=5,
    filter={"category": {"$eq": "shoe"}},
    include_metadata=True
)

paths = ['../data/'+i['metadata']['img_path'] for i in result.matches]

draw_images([Image.open(i) for i in paths])

In [ ]:
input_text = "street fashion"

d = get_single_text_embedding(input_text, model, tokenizer)

result = index.query(
    vector=d[0],
    top_k=10,
    filter={"category": {"$eq": "top, t-shirt, sweatshirt"}},
    include_metadata=True
)

paths = ['../data/'+i['metadata']['img_path'] for i in result.matches]

draw_images([Image.open(i) for i in paths])

In [ ]:
input_text = "Punk Fashion"

d = get_single_text_embedding(input_text, model, tokenizer)

result = index.query(
    vector=d[0],
    top_k=10,
    filter={"category": {"$eq": "top, t-shirt, sweatshirt"}},
    include_metadata=True
)

paths = ['../data/'+i['metadata']['img_path'] for i in result.matches]

draw_images([Image.open(i) for i in paths])

In [ ]:
input_text = "Bohemian Fashion"

d = get_single_text_embedding(input_text, model, tokenizer)

result = index.query(
    vector=d[0],
    top_k=10,
    filter={"category": {"$eq": "top, t-shirt, sweatshirt"}},
    include_metadata=True
)

paths = ['../data/'+i['metadata']['img_path'] for i in result.matches]

draw_images([Image.open(i) for i in paths])

In [ ]:
input_text = "flower patterns, short sleeve"

d = get_single_text_embedding(input_text, model, tokenizer)

result = index.query(
    vector=d[0],
    top_k=10,
    filter={"category": {"$eq": "top, t-shirt, sweatshirt"}},
    include_metadata=True
)

paths = ['../data/'+i['metadata']['img_path'] for i in result.matches]

draw_images([Image.open(i) for i in paths])

- 장점
    - 유명 브랜드, 성별, 옷의 종류, 색갈 등의 카테고리를 매뉴얼하게 정하지 않아도 input으로 정해줄 수 있다

- 한계점
    - attribute들의 단순 결합이기 때문에 옷의 부위 별 특징을 인식하지 못 함
        - 예) blue dots이라고 명시했지만, 파란색 드레스가 유사도에 표현됨
    - 스트릿, 보헤미안 패션 등 추상적인 단어들은 다양한 옷들의 조합임
    (CLIP은 <옷의 특징>-<옷의 사진> pair를 활용하여 학습. 따라서 "스트릿패션"과 같이 패션의 한 카테고리와 매칭이 되지 않는다.)

- 극복 방안
    - Sparse vector를 활용하여 옷의 특징에 weight를 더 주는 search
    - 옷의 특징이 아닌, 보다 추상적인 텍스트가 들어오는 경우, database 전체를 대상으로 search


---

### 2. Image to image search

- CLIP embedding을 활용
    - text와 image를 한 vector space에 함께 표현되어 있지만 Image-to-Image 유사도 측정도이 가능함
    - 또한, fashion dataset에 맞도록 fine-tune되어, 일반 plain CLIP보다 현재 use case에 적합


In [35]:
from image_utils import extract_img_features

In [ ]:
image = Image.open("../data/imaterialist-fashion-2020-fgvc7/test_images/test_image2.jpg")

img_emb = extract_img_features(image, processor, model).tolist()

result = index.query(
    vector=img_emb[0],
    top_k=5,
    filter={"category": {"$eq": "top, t-shirt, sweatshirt"}},
    include_metadata=True
)

paths = ['../data/'+i['metadata']['img_path'] for i in result.matches]

draw_images([Image.open(i) for i in paths])

In [ ]:
image = Image.open("../data/imaterialist-fashion-2020-fgvc7/test_images/test_image.jpg")

img_emb = extract_img_features(image, processor, model).tolist()

result = index.query(
    vector=img_emb[0],
    top_k=5,
    filter={"category": {"$eq": "shirt, blouse"}},
    include_metadata=True
)

paths = ['../data/'+i['metadata']['img_path'] for i in result.matches]

draw_images([Image.open(i) for i in paths])

- 한계점
    - 이미지에는 옷의 색, 사람들의 포즈, 빛 등 다양한 요소들이 모두 포함되어 있기 때문에 옷의 특징들만 선별하여 서치를 진행할 수 없음
    - 즉, 옷의 디테일을 간과할 가능성이 높음
- 극복 방안
    - 이미지로부터 옷의 특징을 텍스트 형식으로 추출, dense vector 또는 sparse vector로 변환하여 서치

### 3. Hybrid search (Dense & sparse vector search)

- splade를 활용하여 각 파트별 특징까지 고려

In [43]:
from splade.splade.models.transformer_rep import Splade
from transformers import AutoTokenizer
import torch

sparse_model_id = 'naver/splade-cocondenser-ensembledistil'

# splade = 'naver/splade-v3'
sparse_model = Splade(sparse_model_id, agg='max')
sparse_model.to('cpu')  # move to GPU if possible
sparse_model.eval()

splade_tokenizer = AutoTokenizer.from_pretrained(sparse_model_id)

In [ ]:
# v-neck
input_text = "orange party dress with long sleeve, v neck"

d = get_single_text_embedding(input_text, model, tokenizer)
# sparse = gen_sparse_vector(input_text, splade_model, splade_tokenizer)

result = index.query(
    vector=d[0],
    top_k=5,
    filter={"category": {"$eq": "dress"}},
    # sparse_vector=sparse,
    include_metadata=True
)

paths = ['../data/'+i['metadata']['img_path'] for i in result.matches]

draw_images([Image.open(i) for i in paths])

In [ ]:
# v-neck
input_text = "orange party dress with long sleeve, v neck"

d = get_single_text_embedding(input_text, model, tokenizer)
sparse = gen_sparse_vector(input_text, splade_model, splade_tokenizer)

result = index.query(
    vector=d[0],
    top_k=5,
    filter={"category": {"$eq": "dress"}},
    sparse_vector=sparse,
    include_metadata=True
)

paths = ['../data/'+i['metadata']['img_path'] for i in result.matches]

draw_images([Image.open(i) for i in paths])

In [48]:

paths = ['../data/'+i['metadata']['img_path'] for i in result.matches]

# draw_images([Image.open(i) for i in paths])

[i['id'] for i in result.matches]

df.loc[df['vdb_id'].isin([i['id'] for i in result.matches]), ['vdb_id', 'ImageId', 'AttributesNames', 'second_AttributesNames']]

,vdb_id,ImageId,AttributesNames,second_AttributesNames
9336,1874544733f9b11f236154d77faf87c9_9336,1874544733f9b11f236154d77faf87c9,"bodycon (dress), symmetrical, pencil, normal w...","boat (neck), elbow-length, set-in sleeve"
59947,9cf94dbf0a5f4bf24233e3eb5bba71fb_59947,9cf94dbf0a5f4bf24233e3eb5bba71fb,"tunic (dress), symmetrical, a-line, normal wai...","v-neck, set-in sleeve, wrist-length, dropped-s..."
78568,cde8b89c650ec766fa35bdf588e64d4e_78568,cde8b89c650ec766fa35bdf588e64d4e,"tunic (dress), symmetrical, circle, high waist...","bishop (sleeve), boat (neck), wrist-length"
79807,d121674ebd3ecfd561821fa9f83ac831_79807,d121674ebd3ecfd561821fa9f83ac831,"tunic (dress), a-line, mini (length), no openi...","v-neck, set-in sleeve, wrist-length"
94373,f701653e4cac2fc6e5a3fb9baa3591e1_94373,f701653e4cac2fc6e5a3fb9baa3591e1,"tea (dress), symmetrical, a-line, high waist, ...","bishop (sleeve), v-neck, wrist-length"


In [47]:
[i['id'] for i in result.matches]

['f701653e4cac2fc6e5a3fb9baa3591e1_94373',
 'd121674ebd3ecfd561821fa9f83ac831_79807',
 '1874544733f9b11f236154d77faf87c9_9336',
 '9cf94dbf0a5f4bf24233e3eb5bba71fb_59947',
 'cde8b89c650ec766fa35bdf588e64d4e_78568']

패션 스타일 관련 text field가 없기 때문에, sparse vector를 활용하더라도 큰 퍼포먼스 향상을 기대하기 어려움

In [ ]:
input_text = "Punk Fashion"

d = get_single_text_embedding(input_text, model, tokenizer)
# sparse = gen_sparse_vector(input_text, splade_model, splade_tokenizer)

result = index.query(
    vector=d[0],
    top_k=10,
    # sparse_vector=sparse,
    filter={"category": {"$eq": "top, t-shirt, sweatshirt"}},
    include_metadata=True
)

paths = ['../data/'+i['metadata']['img_path'] for i in result.matches]

draw_images([Image.open(i) for i in paths])

In [ ]:
input_text = "Punk Fashion"

d = get_single_text_embedding(input_text, model, tokenizer)
sparse = gen_sparse_vector(input_text, splade_model, splade_tokenizer)

result = index.query(
    vector=d[0],
    top_k=10,
    sparse_vector=sparse,
    filter={"category": {"$eq": "top, t-shirt, sweatshirt"}},
    include_metadata=True
)

paths = ['../data/'+i['metadata']['img_path'] for i in result.matches]

draw_images([Image.open(i) for i in paths])

- 활용할 수 있는 attribute의 종류
```python
list_of_attributes = ['main_category', 'silhouette', 'silhouette_fit', 'waistline', 'length',
       'collar_type', 'neckline_type', 'sleeve_type', 'pocket_type',
       'opening_type', 'non-textile material type', 'leather',
       'textile finishing, manufacturing techniques', 'textile pattern']
```
<br>

- attribute으로 표현할 수 있는 document의 포맷

```json
silhouette_name : symmetrical,
collar_type_name : shirt (collar),
opening_type_name : single breasted,
non-textile material type_name : no non-textile material,
textile finishing, manufacturing techniques_name : no special manufacturing technique,
textile pattern_name : plain (pattern)

```

In [ ]:
image = Image.open("test_images/test_image3.png")

img_emb = extract_img_features(image, processor, model).tolist()

result = index.query(
    vector=img_emb,
    top_k=5,  # how many results to return
    filter={"category": {"$eq": "jacket"}},
    include_metadata=True
)

paths = ['../data/'+i['metadata']['img_path'] for i in result.matches]

draw_images([Image.open(i) for i in paths])

In [ ]:
image

In [ ]:
image = Image.open("test_images/test_image3.png")

img_emb = extract_img_features(image, processor, model).tolist()

sparse_vector = gen_sparse_vector("suede jacket", sparse_model, splade_tokenizer)

result = index.query(
    vector=img_emb,
    sparse_vector=sparse_vector,
    top_k=5,  # how many results to return
    filter={"category": {"$eq": "jacket"}},
    include_metadata=True
)

paths = [i['metadata']['img_path'] for i in result.matches]

draw_images([Image.open(i) for i in paths])